In [109]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import numpy as np

from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from pandas.plotting import autocorrelation_plot
import cv2
from skimage.io import imread
from keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle

BATCH_SIZE = 32
EPOCHS = 50
earlystop = EarlyStopping(patience=5)

In [110]:
train = pd.read_csv("../data/processed/train_processed.csv")
df_test = pd.read_csv("../data/processed/test_processed.csv")
test = df_test.drop(["Name", "PassengerId","Cabin"], axis =1)

In [111]:
columnas_bool = train.select_dtypes(include='bool').columns

train[columnas_bool] = train[columnas_bool].astype(int)

columnas_bool = test.select_dtypes(include='bool').columns

test[columnas_bool] = test[columnas_bool].astype(int)

In [112]:
train.dtypes

HomePlanet           int64
CryoSleep            int32
Destination          int64
Age                float64
VIP                  int32
RoomService        float64
FoodCourt          float64
ShoppingMall       float64
Spa                float64
VRDeck             float64
Transported          int32
Spa_VRDeck_0.0       int32
all_0.0              int32
Earth_0.0            int32
TRAPPIST-1e_0.0      int32
dtype: object

In [113]:
X_train, X_test, y_train, y_test = train_test_split(train.drop(["Transported"],axis=1), train["Transported"], test_size=0.2, random_state=42)

In [114]:
print(X_train.shape)

(6954, 14)


In [115]:
layers = [
    keras.layers.Dense(128, activation='relu', input_shape=(14,)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
]


model = keras.Sequential(layers)

model.compile(optimizer = 'adam',
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

In [116]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_56 (Dense)            (None, 128)               1920      
                                                                 
 dense_57 (Dense)            (None, 32)                4128      
                                                                 
 dense_58 (Dense)            (None, 1)                 33        
                                                                 
Total params: 6081 (23.75 KB)
Trainable params: 6081 (23.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [120]:
history = model.fit(X_train,
         y_train,
         epochs = EPOCHS,
         batch_size = BATCH_SIZE,
         callbacks = [earlystop],
         validation_split = 0.2)

Epoch 1/50
174/174 [==============================] - 0s 2ms/step - loss: 0.4314 - accuracy: 0.7999 - val_loss: 0.4190 - val_accuracy: 0.8109
Epoch 2/50
174/174 [==============================] - 0s 1ms/step - loss: 0.4315 - accuracy: 0.8005 - val_loss: 0.4212 - val_accuracy: 0.8160
Epoch 3/50
174/174 [==============================] - 0s 1ms/step - loss: 0.4300 - accuracy: 0.8023 - val_loss: 0.4165 - val_accuracy: 0.8116
Epoch 4/50
174/174 [==============================] - 0s 1ms/step - loss: 0.4299 - accuracy: 0.8010 - val_loss: 0.4249 - val_accuracy: 0.8081
Epoch 5/50
174/174 [==============================] - 0s 1ms/step - loss: 0.4338 - accuracy: 0.7990 - val_loss: 0.4162 - val_accuracy: 0.8145
Epoch 6/50
174/174 [==============================] - 0s 1ms/step - loss: 0.4309 - accuracy: 0.8017 - val_loss: 0.4193 - val_accuracy: 0.8066
Epoch 7/50
174/174 [==============================] - 0s 1ms/step - loss: 0.4329 - accuracy: 0.7976 - val_loss: 0.4110 - val_accuracy: 0.8095
Epoch 

In [121]:
predicciones = model.predict(test)

134/134 [==============================] - 0s 782us/step


In [122]:
predicciones

array([[0.5673081 ],
       [0.01239444],
       [0.9721256 ],
       ...,
       [0.91998065],
       [0.59906745],
       [0.6107827 ]], dtype=float32)

In [129]:
predicciones_umbral = np.where(predicciones < 0.5, False, True)

In [130]:
predicciones_umbral

array([[ True],
       [False],
       [ True],
       ...,
       [ True],
       [ True],
       [ True]])

In [131]:
pasajero = pd.read_csv("../data/raw/test.csv")

In [132]:
submission = pd.DataFrame()
submission["PassengerId"] = pasajero["PassengerId"]
submission["Transported"] = predicciones_umbral

In [133]:
submission

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [134]:
submission.to_csv("../data/processed/Submission_keras.csv", index=False)